In [1]:
import json
import torch
import numpy as np

def get_pdprams(pdparms_path, model):

    pdparams = {}
    with open(pdparms_path, "r") as fp:
        pdparams = dict(json.load(fp))

    with torch.no_grad():
        for name, param in model.named_parameters():
            # 跳过不需要加载的参数
            if name not in pdparams:
                continue
            # 将权重从字典 pdparams 中加载到模型中
            param.copy_(torch.from_numpy(np.array(pdparams[name])))

In [2]:
import torch
    
device = "cuda" if torch.cuda.is_available() else "cpu"

from models import cpm_bn_hand

model = cpm_bn_hand.CPM_hand_torch(21).to(device)

pdprams_path = "./bn_500.json"

get_pdprams(pdprams_path, model)

import cv2

img = cv2.imread("./aa.jpg")

import thirdparty.mytools as mytools
import thirdparty.mytransform as mytransform

size_w = img.shape[1]
size_h = img.shape[0]

 

In [10]:
with torch.no_grad():
      # img = cv2.imread("./aa.jpg")
      img = cv2.imread("./myhand.jpg")
      img = cv2.resize(img, (368, 368))
      import copy
      img0 = copy.deepcopy(img)
      img = np.transpose(img, (2, 0 ,1))

      img = img.astype(np.float32)
      img = torch.from_numpy(img).unsqueeze(0)  # 添加一个维度以匹配模型的期望输入形状
      # centermap = centermap.astype(np.float32)
      centermap = mytools.center_map_default(368, 368, 3)
      centermap = torch.from_numpy(centermap)
      centermap = centermap.unsqueeze(0) # 添加一个维度，变成1 * 1 * H * W
      # centermap = torch.nn.functional.interpolate(centermap, size=(size_h, size_w), mode='bilinear', align_corners=False)

      
      heat1, heat2, heat3, heat4, heat5, heat6 = model(img.to(device), centermap.to(device))
      heatmapi = heat6[0].cpu().numpy()
      kptsi = mytools.get_kpts_from_heatmap(heatmapi, 368., 368.)
      #kptsi_t = get_kpts_from_heatmap(heatmapi_t, 368., 368.)
      # kpts_rmse.append(compute_RMSE(heatmapi, heatmapi_t))
      import copy
      imagei_p = copy.deepcopy(img[0])
      imagei_p = np.transpose(imagei_p, (1, 2, 0))
      # imagei_t = copy.deepcopy(imagei)
      imagei_p = mytools.draw_paint(imagei_p.numpy().copy(), kptsi)
      # imagei_t = draw_paint(imagei_t.copy(), kptsi_t)
      cv2.imwrite(f'1.jpg', imagei_p)
      # cv2.imwrite(f'2.jpg', imagei_t)

In [8]:
imagei_p.shape

torch.Size([368, 368, 3])

In [ ]:
aa = cv2.resize(aa, (368, 368))

In [ ]:
aa.shape

(368, 368, 3)

In [ ]:
img = cv2.imread("/home/louxd/dataset/FreiHand/FreiHAND_pub_v2_eval/evaluation/rgb/00000989.jpg")

In [ ]:
# img = cv2.imread("./aa.jpg")
img = cv2.imread("/home/louxd/dataset/FreiHand/FreiHAND_pub_v2_eval/evaluation/rgb/00000989.jpg")
size_w = img.shape[1]
size_h = img.shape[0]
img = cv2.resize(img, (368, 368))
import copy
img0 = copy.deepcopy(img)

img = np.transpose(img, (2, 0 ,1))

img = img.astype(np.float32)
img = torch.from_numpy(img).unsqueeze(0)  # 添加一个维度以匹配模型的期望输入形状
# centermap = centermap.astype(np.float32)
centermap = mytools.center_map_default(size_w, size_h, 3)
centermap = torch.from_numpy(centermap)
centermap = centermap.unsqueeze(0) # 添加一个维度，变成1 * 1 * H * W
# centermap = torch.nn.functional.interpolate(centermap, size=(size_h, size_w), mode='bilinear', align_corners=False)

In [ ]:
img.shape

torch.Size([1, 3, 368, 368])

In [ ]:
centermap.shape

torch.Size([1, 1, 224, 224])

In [ ]:
size_w

224

In [1]:
import json
import torch
import numpy as np
import cv2
from models import cpm_bn_hand
import thirdparty.mytools as mytools
import thirdparty.mytransform as mytransform
import socket
import argparse
import copy


def get_pdprams(pdparms_path, model):

    pdparams = {}
    with open(pdparms_path, "r") as fp:
        pdparams = dict(json.load(fp))

    with torch.no_grad():
        for name, param in model.named_parameters():
            # 跳过不需要加载的参数
            if name not in pdparams:
                continue
            # 将权重从字典 pdparams 中加载到模型中
            param.copy_(torch.from_numpy(np.array(pdparams[name])))

def load_model():

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = cpm_bn_hand.CPM_hand_torch(21).to(device)
    pdprams_path = "./bn_500.json"
    get_pdprams(pdprams_path, model)
    return model, device

def process_frame(frame, centermap, model, device):

    centermap = copy.deepcopy(mytransform.to_tensor(centermap))
    centermap = centermap.unsqueeze(0) # 添加一个维度，变成1 * 1 * H * W

    img = cv2.resize(frame, (368, 368))
    img = copy.deepcopy(mytransform.normalize(mytransform.to_tensor(img), np.array([0.5, 0.5, 0.5])*255,
                                np.array([1, 1, 1])*255))
    # img = np.transpose(img, (2, 0 ,1))
    # img = img.astype(np.float32)
    img = torch.from_numpy(img).unsqueeze(0)

    with torch.no_grad():
        heat1, heat2, heat3, heat4, heat5, heat6 = model(img.to(device), centermap.to(device))
        heatmapi = heat6[0].cpu().numpy()
        kptsi = mytools.get_kpts_from_heatmap(heatmapi, 368., 368.)
        img0 = np.transpose(img[0], (1, 2, 0)).numpy()
        img0 = img0 * np.array([0.229, 0.224, 0.225], dtype=np.float32)*255 + np.array([0.485, 0.456, 0.406], dtype=np.float32)*255
        imagei_p = mytools.draw_paint(img0.copy(), kptsi)

    return imagei_p


def main(args):

    model, device = load_model()

    # 创建TCP套接字
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    # WSL虚拟机的IP地址和端口号
    wsl_address = (args.wsl_ip, args.port)
    print(f"ip={args.wsl_ip}, port={args.port}")

    try:
        sock.bind(wsl_address)
        print("connect success.")
        BUFFER_SIZE = 655070  # 数据包大小限制

        # 设置了以图像中心为gauss核的centermap
        centermap = mytools.center_map_default(368, 368, 3)
        # centermap = torch.from_numpy(centermap)
        # centermap = centermap.unsqueeze(0) # 添加一个维度，变成1 * 1 * H * W

        count = 0

        while True:
            # 接收数据
            data, _ = sock.recvfrom(BUFFER_SIZE)
            # print("Received:", len(data), "bytes")
            try:
                frame = cv2.imdecode(np.frombuffer(data, dtype=np.uint8), cv2.IMREAD_COLOR)
                cv2.imshow('frame', frame)
                imagei_p = process_frame(frame, centermap, model, device)
                print(imagei_p.shape)
                # 在WSL中显示图像
                cv2.imshow('process img', imagei_p)
                if cv2.waitKey(1) == ord('q'):
                    break
            except Exception as e:
                print("Error decoding image:", e)

            count += 1
            if (count % 100) == 0:
                cv2.imwrite(f'./output/{count}.jpg', imagei_p)

    except Exception as e:
        print("Error:", e)

    finally:
        # 关闭套接字和窗口
        sock.close()
        cv2.destroyAllWindows()


In [3]:
frame = cv2.imread("./aa.jpg")

model, device = load_model()


In [13]:
centermap = mytools.center_map_default(368, 368, 3)
# centermap = copy.deepcopy(mytransform.to_tensor(centermap))
centermap = torch.from_numpy(centermap).unsqueeze(0) # 添加一个维度，变成1 * 1 * H * W

In [14]:
centermap.shape

torch.Size([1, 1, 368, 368])

In [15]:


img = cv2.resize(frame, (368, 368))
img = copy.deepcopy(mytransform.normalize(mytransform.to_tensor(img), np.array([0.5, 0.5, 0.5])*255,
                            np.array([1, 1, 1])*255))
# img = np.transpose(img, (2, 0 ,1))
# img = img.astype(np.float32)
img = img.unsqueeze(0)

with torch.no_grad():
    heat1, heat2, heat3, heat4, heat5, heat6 = model(img.to(device), centermap.to(device))
    heatmapi = heat6[0].cpu().numpy()
    kptsi = mytools.get_kpts_from_heatmap(heatmapi, 368., 368.)
    img0 = np.transpose(img[0], (1, 2, 0)).numpy()
    img0 = img0 * np.array([0.229, 0.224, 0.225], dtype=np.float32)*255 + np.array([0.485, 0.456, 0.406], dtype=np.float32)*255
    imagei_p = mytools.draw_paint(img0.copy(), kptsi)


In [19]:
imagei_p.shape

(368, 368, 3)

In [20]:
cv2.imwrite("1111.jpg", imagei_p)

True